# NEWS 파일(json)을 Bigquery에 저장

In [2]:
import os
import sys
import json
import requests
import pickle
import gzip
import pandas as pd
import pandas_gbq as gbq
from pathlib import Path
from google.oauth2 import service_account
from google.cloud import bigquery
from IPython.core.debugger import set_trace
from tqdm.auto import tqdm
tqdm.pandas()

## Configuration

In [3]:
proj = 'global-news-crawl'
table_downloaded = 'news_dataset.downloaded'
table_trashed = 'news_dataset.trashed'
credentials = service_account.Credentials.from_service_account_file('global-news-crawl-c48d7cd9aa81.json')

## Bigquery에서 기존 뉴스 ID 리스트 받기

In [7]:
%%time
qry_0 = 'SELECT id FROM `{}`'.format(proj + '.' + table_downloaded)
ids_downloaded = gbq.read_gbq(qry_0, project_id=proj, credentials=credentials)#, use_bqstorage_api=True)

Wall time: 11.7 s


In [19]:
qry_1 = 'SELECT id FROM `{}`'.format(proj + '.' + table_trashed)
ids_trashed = gbq.read_gbq(qry_1, project_id=proj, credentials=credentials)

In [20]:
ids_downloaded_set = set(ids_downloaded.id)
ids_trashed_set = set(ids_trashed.id)

In [21]:
assert len(ids_downloaded) == len(ids_downloaded_set), 'duplicated in downloaded'
assert len(ids_trashed) == len(ids_trashed_set), 'duplicated in downloaded'
assert len(ids_downloaded_set & ids_trashed_set) == 0, 'ids overlapping'

In [22]:
newsids = ids_downloaded_set | ids_trashed_set; len(newsids)

371253

In [23]:
len(ids_downloaded_set), len(ids_trashed_set)

(229018, 142235)

## 파일에서 뉴스정보 추출
나중에는 필요없는 과정이다. 다운로드 받은 즉시 바로 Bigquery에 전송할 것이므로

In [24]:
def extract_contents(newsids=None, where=None, n=1000):
    df = {}
    _n = 0
    
    for file in Path(where).glob('**/*.json'):
        id = file.stem
        
        if id not in newsids:
            try:
                js = json.loads(file.read_text())
            
                if 'authors' in js:
                    js['authors'] = ', '.join(js['authors'])

                df[id] = js

                _n += 1
                print('\r{}'.format(_n), end='')
                if _n == n: break
                    
            except:
                print(file)
           
    df = pd.DataFrame.from_dict(df, orient='index')
    df.index.name = 'id'
    return df.reset_index()

In [25]:
df_downloaded = extract_contents(newsids=newsids, where='newsdata/downloaded', n=50000); print('\n')
df_trashed = extract_contents(newsids=newsids, where='newsdata/trashed', n=30000)

2355

1968

## downloaded와 trashed 간에 겹치는 게 없는지 확인
기존의 뉴스파일을 Bigquery에 전송하는 과정에서, 이 둘간에 겹치는 사례가 종종 있었다
(2019.10.18)

In [26]:
intersect = set(df_trashed.id) & set(df_downloaded.id)
assert len(intersect) == 0, 'ids overlapping'

## Bigquery에 업로드

In [27]:
gbq.to_gbq(df_downloaded, table_downloaded, project_id=proj, if_exists='append', chunksize=1000, credentials=credentials)
gbq.to_gbq(df_trashed, table_trashed, project_id=proj, if_exists='append', chunksize=1000, credentials=credentials)

3it [00:20,  6.94s/it]
2it [00:10,  5.34s/it]


In [115]:
class BigqueryLoader:
    def __init__(self):
        self.ids = self._get_ids()
    
    def _get_ids(self):
        print('checking bigqueryloader status... ', end='')
        
        qry_ids = lambda prj, tb: 'SELECT id FROM `{}`'.format(prj + '.' + tb)
        qry_downloaded = qry_ids(proj, table_downloaded)
        qry_trashed = qry_ids(proj, table_trashed)
        
        ids_downloaded = gbq.read_gbq(qry_downloaded, project_id=proj, credentials=credentials)
        ids_trashed = gbq.read_gbq(qry_trashed, project_id=proj, credentials=credentials)
        
        ids_downloaded_set = set(ids_downloaded.id)
        ids_trashed_set = set(ids_trashed.id)
        
        if len(ids_downloaded) != len(ids_downloaded_set):
            raise self.DuplicatesInSingleTable('duplicated in downloaded')

        if len(ids_trashed) != len(ids_trashed_set):
            raise self.DuplicatesInSingleTable('duplicated in trashed')

        if len(ids_downloaded_set & ids_trashed_set) != 0:
            raise self.DuplicatesBetweenTwoTables('duplicated between downloaded and trashed')
    
        ids = ids_downloaded_set | ids_trashed_set
        
        print('done')
        print('we have: total {} ({} downloaded, {} trashed)'.format(len(ids), len(ids_downloaded_set), len(ids_trashed_set)))
        return ids
        
        
    def has(self, id):
        return id in self.ids
        
    def upload(self, df, to, chunksize=1000):
        gbq.to_gbq(df, to, project_id=proj, if_exists='append', chunksize=chunksize, credentials=credentials)
        
    class DuplicatesInSingleTable(Exception):
        pass
    
    class DuplicatesBetweenTwoTables(Exception):
        pass

In [41]:
proj = 'global-news-crawl'
table_downloaded = 'news_dataset.downloaded'
table_trashed = 'news_dataset.trashed'
credentials = service_account.Credentials.from_service_account_file('global-news-crawl-c48d7cd9aa81.json')

localpath_to_downloaded = 'newsdata/downloaded'
localpath_to_trashed = 'newsdata/trashed'


class ContentManager:
    def __init__(self, where='local'):
        self.where = where
        self.ids = self._get_ids(where)
            
            
    def _get_ids(self, where):
        print('checking news ids status... ', end='')

        if where == 'bigquery':
            qry_ids = lambda prj, tb: 'SELECT id FROM `{}`'.format(prj + '.' + tb)
            qry_downloaded = qry_ids(proj, table_downloaded)
            qry_trashed = qry_ids(proj, table_trashed)

            ids_downloaded = gbq.read_gbq(qry_downloaded, project_id=proj, credentials=credentials).id
            ids_trashed = gbq.read_gbq(qry_trashed, project_id=proj, credentials=credentials).id

        elif where == 'local':
            ids_downloaded = [p.stem for p in Path(localpath_to_downloaded).glob('**/*.json')]
            ids_trashed = [p.stem for p in Path(localpath_to_trashed).glob('**/*.json')]


        ids_downloaded_set = set(ids_downloaded)
        ids_trashed_set = set(ids_trashed)
        
        if len(ids_downloaded) != len(ids_downloaded_set):
            raise self.DuplicatesInSingleTable('duplicated in downloaded')

        if len(ids_trashed) != len(ids_trashed_set):
            raise self.DuplicatesInSingleTable('duplicated in trashed')

        if len(ids_downloaded_set & ids_trashed_set) != 0:
            raise self.DuplicatesBetweenTwoTables('duplicated between downloaded and trashed')
    
        ids = ids_downloaded_set | ids_trashed_set
        
        print('done')
        print('we have: total {} ({} downloaded, {} trashed)'.format(len(ids), len(ids_downloaded_set), len(ids_trashed_set)))
        return ids
    
    
    def has(self, id):
        return id in self.ids
        
        
    def save(self, list_downloaded=None, list_trashed=None, chunksize=1000):
        if self.where == 'bigquery':
            self._save_to_bigquery(list_downloaded, list_trashed, chunksize)
        
        elif self.where == 'local':
            self._save_to_local(list_downloaded, list_trashed)
            
            
    def _save_to_local(self, list_downloaded, list_trashed):
        if list_downloaded is not None:
            for article in list_downloaded:
                id = article.pop('id')
                _dir = Path(localpath_to_downloaded)
                _dir.mkdir(parent=True, exist_ok=True)
                fname = id + '.json'
                fpath = _dir / fname
                with fpath.open('w') as f:
                    json.dump(article)
        
        if list_trashed is not None:
            for article in list_trashed:
                id = article.pop('id')
                _dir = Path(localpath_to_trashed + '/' + id[:3])
                _dir.mkdir(parent=True, exist_ok=True)
                fname = id + '.json'
                fpath = _dir / fname
                with fpath.open('w') as f:
                    json.dump(article)        
            
            
    def _save_to_bigquery(self, list_downloaded, list_trashed, chunksize):
        if list_downloaded is not None:
            df0 = pd.DataFrame(list_downloaded)
            gbq.to_gbq(df0, table_downloaded, project_id=proj, if_exists='append', chunksize=chunksize, credentials=credentials)

        if list_trashed is not None:
            df1 = pd.DataFrame(list_trashed)
            gbq.to_gbq(df1, table_trashed, project_id=proj, if_exists='append', chunksize=chunksize, credentials=credentials)
                
    
    class DuplicatesInSingleTable(Exception):
        pass
    
    class DuplicatesBetweenTwoTables(Exception):
        pass

In [42]:
cmanager = ContentManager(where='local')

checking news ids status... done
we have: total 378662 (233098 downloaded, 145564 trashed)


In [32]:
p = Path("newsdata/test")
p.mkdir(parents=True, exist_ok=True)
fname = "test.json"
fpath = p / fname
with fpath.open("w") as f:
    json.dump({'a':1, 'b':2}, f)

In [40]:
for file in Path('newsdata/downloaded').glob('**/*.json'):
    fname = file.name
    

> <ipython-input-40-7480e5057ae0>(1)<module>()
----> 1 for file in Path('newsdata/downloaded').glob('**/*.json'):
      2     fname = file.name
      3     set_trace()

ipdb> fname
'000104e3e10318e0fae25e96eaabece27d88c56a.json'
ipdb> q


BdbQuit: 